In [40]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn import svm

from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import BaggingClassifier

import eli5

# загружаем данные

In [6]:
data = pd.read_csv('titanic.csv')

In [7]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


# Предварительная работа с данными 

In [8]:
columns_target = ['Survived'] #целевая колонка

In [9]:
columns_train = ['Pclass', 'Sex', 'Age', 'Fare']

In [10]:
Y = data[columns_target]
X = data[columns_train]

# Проверим естть ли пустые ячейки

In [11]:
print(X['Sex'].isnull().sum(), X['Pclass'].isnull().sum(), X['Fare'].isnull().sum(), X['Age'].isnull().sum()) 

0 0 0 177


# Заполняем пустые значения медианным значением

In [12]:
X['Age'] = X['Age'].fillna(X['Age'].median())

/var/folders/9y/1_m0916n3f926sbs3ljhhd300000gn/T/ipykernel_32898/1341152632.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Age'] = X['Age'].fillna(X['Age'].median())


In [266]:
X['Age'].loc[850:900]

850     4.0
851    74.0
852     9.0
853    16.0
854    44.0
855    18.0
856    45.0
857    51.0
858    24.0
859    28.0
860    41.0
861    21.0
862    48.0
863    28.0
864    24.0
865    42.0
866    27.0
867    31.0
868    28.0
869     4.0
870    26.0
871    47.0
872    33.0
873    47.0
874    28.0
875    15.0
876    20.0
877    19.0
878    28.0
879    56.0
880    25.0
881    33.0
882    22.0
883    28.0
884    25.0
885    39.0
886    27.0
887    19.0
888    28.0
889    26.0
890    32.0
Name: Age, dtype: float64

In [13]:
X['Age'].isnull().sum()

0

# Заменяем male и famale в 0 и 1

In [14]:
d={'male':0,'female':1}

In [15]:
X['Sex'] = X['Sex'].apply(lambda x:d[x])

/var/folders/9y/1_m0916n3f926sbs3ljhhd300000gn/T/ipykernel_32898/2841940904.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Sex'] = X['Sex'].apply(lambda x:d[x])


In [16]:
X['Sex'].head(5)

0    0
1    1
2    1
3    1
4    0
Name: Sex, dtype: int64

# Разделяем нашу выборку на тестовую и тренировочную 

In [51]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.1,random_state=42)

In [53]:
clf = tree.DecisionTreeClassifier (max_depth=5, random_state= 21) #

In [55]:
bagging = BaggingClassifier(base_estimator=clf, n_estimators=100)

In [56]:
bagging.fit(X_train,Y_train)

/Users/vladimirsemenov/anaconda3/lib/python3.10/site-packages/sklearn/ensemble/_bagging.py:802: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vladimirsemenov/anaconda3/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=5,
                                                        random_state=21),
                  n_estimators=100)

In [57]:
bagging.score(X_test,Y_test)

0.8333333333333334

In [58]:
clf.fit(X_train,Y_train)

DecisionTreeClassifier(max_depth=5, random_state=21)

In [59]:
clf.score(X_test,Y_test)

0.7666666666666667

In [39]:
eli5.explain_weights_sklearn(clf, feature_names=X_train.columns.values)
#проверка на то что имеет больший вес в датасете с большего к меньньшему

Weight,Feature
0.5264,Sex
0.1795,Pclass
0.1749,Fare
0.1192,Age


# Загружаем модель Support Vector Machine

In [27]:
predmodel = svm.LinearSVC()

In [28]:
predmodel.fit(X_train,Y_train)

/Users/vladimirsemenov/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/vladimirsemenov/anaconda3/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC()

In [29]:
#Предсказываем на тестовой выборке
predmodel.predict(X_test[0:10])

array([0, 0, 0, 1, 1, 1, 1, 0, 1, 1])

In [30]:
#Проверяем точность предсказаний 
predmodel.score(X_test,Y_test)

0.8169491525423729